In [1]:
#Libraries
import pandas as pd
from sqlalchemy import create_engine
import datetime

Team Members:
Meliha Osmanovic, Ashley J Showers, Laylaa Tariq
The sources that we used to extract our data from are:
1. Kaggle
2. Google dataset Search

We filtered out the columns as per our needs. Using postgres database to create tables and load data into the tables. Hot_stuff dataset included 320496 rows and we extracted data for 2019 approximately 5300 rows. Top 50 dataset has only the top 50 songs on spotify, and we extracted the relevant columns.

We ran into 2 problems where we had not psycop2 not installed on some of the computers so had trouble estabshling connection to the database. Another one was we named our database and one of the tables same and was not working. 

# Extract

In [2]:
#Reading the Weekly Top 100 songs csv
hot_stuff = "hot_stuff_2.csv"
hot_stuff_df = pd.read_csv(hot_stuff)

#Reading the Top 50 songs on spootify csv
top_50 = "top50.csv"
top_50_df = pd.read_csv(top_50, encoding = "ISO-8859-1")

# Transformation

In [3]:
#Creating a filtered DataFrane from specific Columns
hot_col = ["weekid", "song", "week_position", "performer"]
hot_stuff_transformed = hot_stuff_df[hot_col].copy()


In [4]:
#Converting the weekid Column to Datetime format
hot_stuff_transformed['weekid'] = pd.to_datetime(hot_stuff_transformed['weekid'], format='%m/%d/%Y')

hot_stuff_transformed.head()

,weekid,song,week_position,performer
0,2019-12-28,All I Want For Christmas Is You,1,Mariah Carey
1,2019-12-28,Rockin' Around The Christmas Tree,2,Brenda Lee
2,2019-12-28,Circles,3,Post Malone
3,2019-12-28,Roxanne,4,Arizona Zervas
4,2019-12-28,Futsal Shuffle 2020,5,Lil Uzi Vert


In [5]:
#Filtering out the rows for year 2019 from the dataset
start_date = "2019-01-05"
end_date  = "2019-12-28"
mask = (hot_stuff_transformed['weekid'] >= start_date) & (hot_stuff_transformed['weekid'] <= end_date)
hot_stuff_transformed = hot_stuff_transformed.loc[mask]


In [6]:
#Setting Index for this dataframe
hot_stuff_transformed.set_index('weekid', inplace=True)
hot_stuff_transformed

,song,week_position,performer
weekid,,,
2019-12-28,All I Want For Christmas Is You,1,Mariah Carey
2019-12-28,Rockin' Around The Christmas Tree,2,Brenda Lee
2019-12-28,Circles,3,Post Malone
2019-12-28,Roxanne,4,Arizona Zervas
2019-12-28,Futsal Shuffle 2020,5,Lil Uzi Vert
...,...,...,...
2019-01-05,Last Shot,96,Kip Moore
2019-01-05,Fine China,97,Future & Juice WRLD
2019-01-05,Envy Me,98,Calboy


In [7]:
#Transforming top 50 songs dataframe
col = ["rating", "track", "artist", "genre"]
top_50_df = top_50_df[col].copy()


# Rename the column headers
top_50_df = top_50_df.rename(columns={"rating": "Rank",
                                      "track": "track_name",
                                      "artist": "artist_name"})

In [8]:
#Setting the Index
top_50_df.set_index('Rank', inplace=True)

top_50_df.head()

,track_name,artist_name,genre
Rank,,,
1,Señorita,Shawn Mendes,canadian pop
2,China,Anuel AA,reggaeton flow
3,boyfriend (with Social House),Ariana Grande,dance pop
4,Beautiful People (feat. Khalid),Ed Sheeran,pop
5,Goodbyes (Feat. Young Thug),Post Malone,dfw rap


# Create Database Connection

In [9]:
#Establsihing a connection to postgres
connection_string = "postgres:postgres@localhost:5432/top_50"
engine = create_engine(f'postgresql://{connection_string}')

In [10]:
#Names of the tables
engine.table_names()

['hot_stuff', 'Top_50']

# Load data to the Database

In [11]:
#Pushing data into the tables
hot_stuff_transformed.to_sql(name='hot_stuff', con=engine, if_exists='append', index=True)
top_50_df.to_sql(name='Top_50', con=engine, if_exists='append', index=True)